In [1]:
%%capture --no-stderr
%pip install --quiet -U faker

In [2]:
import os

# Load environment variables for Kafka and VastDB connectivity
DOCKER_HOST_OR_IP = os.getenv("DOCKER_HOST_OR_IP", "localhost")
VASTDB_ENDPOINT = os.getenv("VASTDB_ENDPOINT")
VASTDB_ACCESS_KEY = os.getenv("VASTDB_ACCESS_KEY")
VASTDB_SECRET_KEY = os.getenv("VASTDB_SECRET_KEY")

VASTDB_FRAUD_DETECTION_BUCKET = os.getenv("VASTDB_FRAUD_DETECTION_BUCKET")
VASTDB_FRAUD_DETECTION_SCHEMA = os.getenv("VASTDB_FRAUD_DETECTION_SCHEMA")
VASTDB_FRAUD_DETECTION_TABLE = 'fraud'

# Kafka broker configuration using environment variable
DOCKER_HOST_OR_IP = os.getenv("DOCKER_HOST_OR_IP")
print(f"{DOCKER_HOST_OR_IP=}")

kafka_brokers = f"{DOCKER_HOST_OR_IP}:19092"
kafka_topic = "stock-settlement"

DOCKER_HOST_OR_IP='10.143.11.241'


## Clean up Database

In [3]:
import os
import vastdb

# Print configurations
print(f"""
---
DOCKER_HOST_OR_IP={DOCKER_HOST_OR_IP}
---
VASTDB_ENDPOINT={VASTDB_ENDPOINT}
VASTDB_ACCESS_KEY={VASTDB_ACCESS_KEY[-4:]}
VASTDB_SECRET_KEY=****{VASTDB_SECRET_KEY[-4:]}
VASTDB_FRAUD_DETECTION_BUCKET={VASTDB_FRAUD_DETECTION_BUCKET}
VASTDB_FRAUD_DETECTION_SCHEMA={VASTDB_FRAUD_DETECTION_SCHEMA}
VASTDB_FRAUD_DETECTION_TABLE={VASTDB_FRAUD_DETECTION_TABLE}
---
""")

def drop_vastdb_table(endpoint, access_key, secret_key, bucket_name, schema_name, table_name):
    session = vastdb.connect(endpoint=endpoint, access=access_key, secret=secret_key)
    with session.transaction() as tx:
        try:
            table = tx.bucket(bucket_name).schema(schema_name).table(table_name)
            if table: 
                table.drop()
                print(f"Table {table} dropped")
        except Exception as e:
            print("Bucket, Schema or Table not found")
            return


---
DOCKER_HOST_OR_IP=10.143.11.241
---
VASTDB_ENDPOINT=http://172.200.204.2:80
VASTDB_ACCESS_KEY=QXN5
VASTDB_SECRET_KEY=****oLGr
VASTDB_FRAUD_DETECTION_BUCKET=csnow-db
VASTDB_FRAUD_DETECTION_SCHEMA=fraud_detection
VASTDB_FRAUD_DETECTION_TABLE=fraud
---



In [4]:
drop_vastdb_table(
    VASTDB_ENDPOINT, 
    VASTDB_ACCESS_KEY, 
    VASTDB_SECRET_KEY, 
    VASTDB_FRAUD_DETECTION_BUCKET,
    VASTDB_FRAUD_DETECTION_SCHEMA,
    VASTDB_FRAUD_DETECTION_TABLE
)

Bucket, Schema or Table not found


# Clean up Kafka

In [5]:
%%capture --no-stderr
%pip install kafka-python six kafka-python-ng

In [6]:
import os

kafka_brokers = f"{DOCKER_HOST_OR_IP}:19092"
kafka_topic = "stock-settlement"

# Print configurations
print(f"""
---
DOCKER_HOST_OR_IP={DOCKER_HOST_OR_IP}
---
VASTDB_ENDPOINT={VASTDB_ENDPOINT}
VASTDB_ACCESS_KEY={VASTDB_ACCESS_KEY[-4:]}
VASTDB_SECRET_KEY=****{VASTDB_SECRET_KEY[-4:]}
VASTDB_FRAUD_DETECTION_BUCKET={VASTDB_FRAUD_DETECTION_BUCKET}
VASTDB_FRAUD_DETECTION_SCHEMA={VASTDB_FRAUD_DETECTION_SCHEMA}
# VASTDB_FRAUD_DETECTION_TABLE={VASTDB_FRAUD_DETECTION_TABLE}
---
""")

from kafka.admin import KafkaAdminClient, NewTopic

# Kafka connection parameters
bootstrap_servers = f"{DOCKER_HOST_OR_IP}:19092"
topic_name = "stock-settlement"

# Create Kafka Admin client
admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers,
    client_id='cleanup-topic'
)

# Delete the topic
try:
    admin_client.delete_topics([topic_name])
    print(f"Topic '{topic_name}' deleted successfully.")
except Exception as e:
    print(f"Failed to delete topic: {e}")

# Optionally, recreate the topic
try:
    new_topic = NewTopic(name=topic_name, num_partitions=1, replication_factor=1)
    admin_client.create_topics([new_topic])
    print(f"Topic '{topic_name}' recreated successfully.")
except Exception as e:
    print(f"Failed to recreate topic: {e}")


---
DOCKER_HOST_OR_IP=10.143.11.241
---
VASTDB_ENDPOINT=http://172.200.204.2:80
VASTDB_ACCESS_KEY=QXN5
VASTDB_SECRET_KEY=****oLGr
VASTDB_FRAUD_DETECTION_BUCKET=csnow-db
VASTDB_FRAUD_DETECTION_SCHEMA=fraud_detection
# VASTDB_FRAUD_DETECTION_TABLE=fraud
---

Topic 'stock-settlement' deleted successfully.
Topic 'stock-settlement' recreated successfully.
